In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import optimize

from sklearn.metrics import accuracy_score, f1_score
from pymoo.core.problem import Problem
from evoman.environment import Environment
from fitness_functions import original_fitness, individual_gain
from utils import simulation, verify_solution, init_env, run_pymoo_algorithm, initialise_script
import math


pygame 2.5.1 (SDL 2.28.2, Python 3.9.17)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
solutions = []
for file in os.listdir('farmed_beats_8'):
    new_solution = []
    with open(f'farmed_beats_8/{file}') as f:
        solutions.append(f.read().splitlines())
            
solutions = np.array(solutions, dtype=float)

In [3]:
POP_SIZE = 1
ENEMIES = [1, 2, 3, 4, 5, 6, 7, 8]
n_hidden_neurons = 10

In [4]:
class objectives(Problem):
    enemies: list[int]
    env: Environment

    def __init__(self, env: Environment, n_genes: int, enemies: list[int], n_objectives):
        self.env = env
        self.enemies = enemies
        super().__init__(n_var=n_genes, n_obj=n_objectives, xl=-1, xu=1, type_var=float)

    def _evaluate(self, x: list[np.array], out, *args, **kwargs):
        #if POP_SIZE != len(x):
        #    print(f"WARNING: POP_SIZE != len(x) in evaluation step (this happens sometimes do not see why)\n"
        #          f"pop size: {POP_SIZE}; len x:{len(x)}")
        # Initialize
        dict_enemies = {}
        # Get fitness for each enemy
        for enemy in self.enemies:
            self.env.update_parameter('enemies', [enemy])

            dict_enemies[enemy] = []
            for individual_id in range(len(x)):
                dict_enemies[enemy].append(
                    simulation(self.env, x[individual_id], inverted_fitness=True, fitness_function=individual_gain))

        # Return fitness outputs for enemies
        objectives_fitness = {
            "objective_hard": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [1, 6]]) for ind_id in
                               range(len(x))],
            "objective_medium": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [2, 5, 8]]) for ind_id in
                                 range(len(x))],
            "objective_easy": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [3, 4, 7]]) for ind_id in
                               range(len(x))],
        }

        out["F"] = np.column_stack([objectives_fitness[key] for key in objectives_fitness.keys()])
        return out

In [5]:
env, n_genes = init_env('local_search_test', ENEMIES, n_hidden_neurons)
env.update_parameter('multiplemode', 'no')
problem = objectives(
        env=env,
        n_genes=n_genes,
        enemies=ENEMIES,
        n_objectives=3)

In [38]:
x_curr = solutions[[253],:]
x_best = np.copy(x_curr)

f_curr = problem._evaluate(x_curr, {})['F']
f_best, f_best_orig = np.copy(f_curr), np.copy(f_curr)

n_nbrs = 25

print(f_best_orig)

[[0.00325098 0.0031085  0.00326986]]


In [6]:
# This is the function that needs to be defined, this is how you find neighbors of solution x
# For now, it gives a neighborhood of n neighbors where every neighbor's gene has a SMALL chance to be altered SLIGHTLY.
# I could not find clear examples of algorithms for the neighbor generating functions, so this will have to do for now

def neighborhood(x, n, p_mut=0.025, sigma=0.1):
    nbrhood = []
    for _ in range(n):
        # Make new copy of solution
        new_nbr = np.copy(x)
        
        # Make probability vector for 'mutating'
        p = np.random.uniform(size=new_nbr.shape[1])
        
        # 'Mutate' genes
        new_nbr[0][p < p_mut] = new_nbr[0][p < p_mut] * np.random.normal(0, sigma)     
        
        # Add neighbor to neighborhood
        nbrhood.append(new_nbr)
    
    return nbrhood

In [ ]:
done = False
solutionsChecked = 0
method = 'steepest ascent'
m = 0
# Steepest ascent local search searches until no better neighbor is found
# First ascent local search searches until the first better neighbor is found
while not done:
    
    # Make new neighborhood
    Neighborhood = neighborhood(x_curr, 25)
    
    # Evaluate every neighbor s
    for s in Neighborhood:
        solutionsChecked += 1
        
        try:
            eval_s = problem._evaluate(s, {})['F']
        except:
            # print("Infeasible solution evaluated")
            continue
        
        # If the neighbor has a better fitness evaluation, this becomes the new best
        if eval_s[0].mean() < f_best[0].mean():
            x_best = np.copy(s)
            f_best = np.copy(eval_s)
    
    # Run until no further improvements (steepest ascent)
    if np.array_equal(f_best, f_curr):
        if m >= 20:
            done = True
        else:
            m += 1
    else:
        x_curr = np.copy(x_best)
        f_curr = np.copy(f_best)
        
    print(f"Total number of solutions checked: {solutionsChecked}")
    print(f"Best value found so far: {f_best} ({1 / f_best})")
        
print(f"Final number of solutions checked: {solutionsChecked}")
print(f"New solution inverted fitness: {f_best} ({1 / f_best})")
print(f"Initial solution fitness: {f_best_orig} ({1 / f_best_orig})")

In [18]:
from tqdm import trange

x_curr = solutions[[253],:]
f_curr = problem._evaluate(x_curr, {})['F']
i_curr = 0

x_best, f_best, i_best = np.copy(x_curr), np.copy(f_curr), i_curr

# First selecting best solution so far
# for i in trange(1, len(solutions)):
#     x_curr = solutions[[i],:]
#     f_curr = problem._evaluate(x_curr, {})['F']
#     # print(f'Evaluating solution {i}: {f_curr[0].mean()}')
    
#     if f_curr[0].mean() <= f_best[0].mean():
#         x_best, f_best, i_best = np.copy(x_curr), np.copy(f_curr), i
        
print(f_best, i_best)

[[0.00325098 0.0031085  0.00326986]] 0


In [40]:
t = 50000  #setting an initial temperature
M = 50    #number of iterations at each temperature

x_curr = solutions[[253],:]
x_best = np.copy(x_curr)

f_curr = problem._evaluate(x_curr, {})['F']
f_best, f_best_orig = np.copy(f_curr), np.copy(f_curr)

solutionsChecked = 0
done = False

# Same idea as steepest ascent, but if a neighbor is not better than the original solution, it still has a chance
# to become the new best solution based on some probability p.
# the rate at which this happens is controlled by the temperature T.
while not done:
    if t < 1:
        done = True
        
    m = 0
    while m < M:
        solutionsChecked += 1
        # print(f"k = {k}, m = {m}, s = {solutionsChecked} \n")
        
        N = neighborhood(x_curr, M)
        idx = np.random.randint(len(N))
        s = N[idx]        
        
        try:
            eval_s = problem._evaluate(s, {})['F']
        except Exception as e:
            break
        
        if eval_s[0].mean() <= f_best[0].mean():
            x_best = np.copy(s)
            f_best = np.copy(eval_s)
        else:
            # In literature, the 1e-7 isn't there but if you leave it out, p=1 for almost every case, so useless
            p = np.exp(-(eval_s[0].mean() - f_best[0].mean()) / (t*1e-7))
            test_p = np.random.uniform(0, 1)
            if test_p <= p:
                x_best = np.copy(s)
                f_best = np.copy(eval_s)
        
        m += 1
    
    t = 0.8*t  #cauchy cooling function
    print(f"s= {solutionsChecked}, t= {t}")
    
print(f"Final number of solutions checked: {solutionsChecked}")
print(f"New solution inverted fitness: {f_best} ({1 / f_best})")
print(f"Initial solution fitness: {f_best_orig} ({1 / f_best_orig})")

s= 50, t= 40000.0
s= 100, t= 32000.0
s= 150, t= 25600.0
s= 200, t= 20480.0
s= 250, t= 16384.0
s= 300, t= 13107.2
s= 350, t= 10485.760000000002
s= 400, t= 8388.608000000002
s= 450, t= 6710.886400000002
s= 500, t= 5368.709120000002
s= 550, t= 4294.967296000002
s= 600, t= 3435.9738368000017
s= 650, t= 2748.7790694400014
s= 700, t= 2199.023255552001
s= 750, t= 1759.218604441601
s= 800, t= 1407.3748835532808
s= 850, t= 1125.8999068426247
s= 900, t= 900.7199254740998
s= 950, t= 720.5759403792799
s= 1000, t= 576.460752303424
s= 1050, t= 461.16860184273924
s= 1100, t= 368.9348814741914
s= 1150, t= 295.14790517935313
s= 1200, t= 236.1183241434825
s= 1250, t= 188.89465931478603
s= 1300, t= 151.11572745182883
s= 1350, t= 120.89258196146307
s= 1400, t= 96.71406556917046
s= 1450, t= 77.37125245533637
s= 1500, t= 61.8970019642691
s= 1550, t= 49.517601571415284
s= 1600, t= 39.61408125713223
s= 1650, t= 31.691265005705784
s= 1700, t= 25.35301200456463
s= 1750, t= 20.282409603651704
s= 1800, t= 16.2259

0.0